## Simple Neural Network

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Load Dataset

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  # y labels are oh-encoded

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Data preparation

In [3]:
n_train = mnist.train.num_examples  # 55,000
n_validation = mnist.validation.num_examples  # 5000
n_test = mnist.test.num_examples  # 10,000

In [4]:
n_input = 784  # input layer (28x28 pixels)
# n_hidden1 = 512  # 1st hidden layer
# n_hidden2 = 256  # 2nd hidden layer
# n_hidden3 = 128  # 3rd hidden layer
n_output = 10  # output layer (0-9 digits)

In [5]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

In [6]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

### cara 1

In [7]:
# weights = {
#     'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
#     'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
#     'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
#     'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
# }

In [8]:
# biases = {
#     'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
#     'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
#     'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
#     'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
# }

In [9]:
# layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
# layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
# layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
# layer_drop = tf.nn.dropout(layer_3, keep_prob)
# output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

### cara 2

In [10]:
# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

In [11]:
# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(X, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

In [12]:
# now calculate the hidden layer output - in this case, let's use a softmax activated
# output layer
output_layer = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

In [13]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
        ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [14]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [16]:
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
        X: batch_x, Y: batch_y, keep_prob: dropout
        })

    # print loss and accuracy (per minibatch)
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
            )
        print(
            "Iteration",
            str(i),
            "\t| Loss =",
            str(minibatch_loss),
            "\t| Accuracy =",
            str(minibatch_accuracy)
            )

Iteration 0 	| Loss = 2.295102 	| Accuracy = 0.1640625
Iteration 100 	| Loss = 2.1212955 	| Accuracy = 0.5390625
Iteration 200 	| Loss = 1.8516973 	| Accuracy = 0.7265625
Iteration 300 	| Loss = 1.8192923 	| Accuracy = 0.765625
Iteration 400 	| Loss = 1.8212367 	| Accuracy = 0.6953125
Iteration 500 	| Loss = 1.7793 	| Accuracy = 0.71875
Iteration 600 	| Loss = 1.6902881 	| Accuracy = 0.84375
Iteration 700 	| Loss = 1.7096441 	| Accuracy = 0.796875
Iteration 800 	| Loss = 1.6687572 	| Accuracy = 0.828125
Iteration 900 	| Loss = 1.674547 	| Accuracy = 0.8359375


In [17]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.8288


In [18]:
from IPython.display import Image
Image(filename='test_img.png', width=300, height=300)

In [19]:
import numpy as np
from PIL import Image

img = np.invert(Image.open("test_img.png").convert('L')).ravel()

In [20]:
prediction = sess.run(tf.argmax(output_layer, 1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 2


## thank you<br><br>

$\textbf{Arif Romadhan}$ <br>
email : arif.romadhan@bukalapak.com<br>
telegram : @SiAnakBatu<br>

reference :<br> 
https://adventuresinmachinelearning.com/python-tensorflow-tutorial/<br>
https://pythonprogramminglanguage.com/tensorflow-neural-network/<br>
https://www.digitalocean.com/community/tutorials/how-to-build-a-neural-network-to-recognize-handwritten-digits-with-tensorflow<br>


